<a id="top"></a>
# Searching MAST using astroquery.mast's Mast class.
***
## Learning Goals

By the end of this tutorial, you will:

- Recognize the types of services MAST Portal API provides through `astroquery.mast` and their required parameters.
- Understand the benefits of `astroquery.mast`'s `Mast` class.
- Make queries for different types of services using `astroquery.mast.Mast.mast_query`.


## Introduction
`astroquery.mast.Mast` is a class that allows programmatic access to the MAST Portal. It is similar to `astroquery.mast.Observations` in that the Observations class also allows direct programmatic access to the MAST Portal. The Mast class is still needed in addition to Observations because Observations provides simplified methods to query MAST's CAOM database, while Mast can be used to query other MAST databases including CAOM. Compared to Observations, using Mast, users can create more refined queries using science instrument keywords. 

Previously, users who wanted to query for observations using the Mast class had to compile their own JSON/dictionaries to feed into the MAST Portal API. To simplify queries, users should use the `mast_query` method under `astroquery.mast.Mast`. 


The workflow for this notebook consists of:
* [Filtered Queries](#Filtered-Queries)
    * [JWST Query](#JWST-Query)
    * [TESS Query](#TESS-Query)
* [Cone Search](#Cone-Search)
* [Mast vs Observations](#Mast-vs-Observations)
* [Additional Resources](#Additional-Resources)

## Imports

- *astropy.units* to handle unit conversion.
- *astropy.time Time* to handle time format conversion.

In [1]:
import astropy.units as u

from astropy.time import Time
from astroquery.mast import Mast
from astroquery.mast import Observations

## Date Format Helper Method

In [2]:
def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }

***

## Filtered Queries

MAST's Filtered services use the parameters 'columns' and 'filters'. The 'columns' parameter is a required string that specifies the columns to be returned as a comma-separated list. The 'filters' parameter is a required list of filters to be applied. Using the `mast_query` method, users can list filters using keyword arguments and a list of values. The examples below will show the necessary format of these arguments for different services.

### JWST Query 

This is a query to view all targets of opportunity observed in the first half of 2023 by JWST's Nirspec instrument. The table only shows the specified columns.

In [3]:
jwst_table = Mast.mast_query('Mast.Jwst.Filtered.Nirspec',
                        columns = 'title, instrume, targname, date_obs_mjd',
                        targoopp = ['T'],
                        date_obs_mjd = [set_mjd_range('2023-01-01', '2023-06-01')])

To verify that the query above returns the correct range of observations, convert the time format form mjd to iso for readability.

In [4]:
jwst_table['date_obs_mjd'] = Time(jwst_table['date_obs_mjd'], format='mjd').iso

In [5]:
# Output the table.
jwst_table

title,instrume,targname,date_obs_mjd
str114,str7,str15,str23
Near- and Mid-IR Observations to Probe Dust Formation in the Remarkably Nearby Stripped-Envelope Supernova 2023dbc,NIRSPEC,SN 2023dbc,2023-05-03 18:18:16.473
Near- and Mid-IR Observations to Probe Dust Formation in the Remarkably Nearby Stripped-Envelope Supernova 2023dbc,NIRSPEC,SN 2023dbc,2023-05-03 18:18:16.473
Near- and Mid-IR Observations to Probe Dust Formation in the Remarkably Nearby Stripped-Envelope Supernova 2023dbc,NIRSPEC,SN 2023dbc,2023-05-03 18:20:00.923
Near- and Mid-IR Observations to Probe Dust Formation in the Remarkably Nearby Stripped-Envelope Supernova 2023dbc,NIRSPEC,SN 2023dbc,2023-05-03 18:20:00.923
Near- and Mid-IR Observations to Probe Dust Formation in the Remarkably Nearby Stripped-Envelope Supernova 2023dbc,NIRSPEC,SN 2023dbc,2023-05-03 18:23:54.223
Near- and Mid-IR Observations to Probe Dust Formation in the Remarkably Nearby Stripped-Envelope Supernova 2023dbc,NIRSPEC,SN 2023dbc,2023-05-03 18:23:54.223
...,...,...,...
NAOMY-J: NIR-dark Accretion Outbursts from Massive Young stellar objects with JWST,NIRSPEC,IRAS 18134-1942,2023-05-07 18:18:55.896
NAOMY-J: NIR-dark Accretion Outbursts from Massive Young stellar objects with JWST,NIRSPEC,IRAS 18134-1942,2023-05-07 18:18:55.896


### TESS Query

TESS queries have 2 types of filtered services. There are [Mast.Catalogs.Filtered...](https://mast.stsci.edu/api/v0/_services.html#MastCatalogsFilteredTic) and [Mast.Catalogs.Filtered...Rows](https://mast.stsci.edu/api/v0/_services.html#MastCatalogsFilteredTicRows) services. To specify columns using the columns parameter in the `mast_query` method, 'rows' services must be used and the other must be used to count the number of returned methods. 

In [6]:
# Example of a 'rows' service
Mast.mast_query('Mast.Catalogs.Filtered.Tic.Position.Rows',
           columns = 'id, objtype, lumclass, ra, dec, Teff, logg',
           Teff = [{'min': 4250, 'max': 4500}], 
           logg = [{'min': 4.5, 'max': 5.0}],
           ra = 210.8023,
           dec = 54.349,
           radius = .2)

id,objtype,lumclass,ra,dec,Teff,logg
str9,str4,str5,float64,float64,float64,float64
441660223,STAR,DWARF,210.637077874118,54.4478798585702,4376.0,4.70571
441660183,STAR,DWARF,210.538862106365,54.2729869423289,4368.0,4.62257
441662171,STAR,DWARF,210.925849073183,54.3818325006587,4281.0,4.67429
441660233,STAR,DWARF,210.632755491987,54.4886965046062,4465.0,4.72413
441662352,STAR,DWARF,211.05020975977,54.3724607057412,4327.0,4.66134
441662136,STAR,DWARF,210.7425480397,54.5405016569961,4369.96,4.50826
441662201,STAR,DWARF,210.775735695348,54.2851241294785,4437.0,4.72054
441662211,STAR,DWARF,210.843392261145,54.259935920806,4339.5,4.66932
441662375,STAR,DWARF,210.956407009065,54.4609598428694,4487.0,4.52807


In [7]:
# Example of a service without 'rows'
Mast.mast_query('Mast.Catalogs.Filtered.Tic.Position',
           columns = 'COUNT_BIG(*)',
           Teff = [{'min': 4250, 'max': 4500}],
           logg = [{'min': 4.5, 'max': 5.0}],
           ra = 210.8023,
           dec = 54.349,
           radius = .2)

Column1
str1
9


## Cone Search

The MAST Portal API's cone searches are not filtered using non-positional criteria. Most cone searches only accept the parameters 'ra', 'dec', and radius and return a table with all columns present. Users can limit the columns by assigning the table returned from the `mast_query` call to a variable and masking it with a list of column names.

In [8]:
gaia_table = Mast.mast_query('Mast.Catalogs.GaiaDR1.Cone',
                        ra = 254.287,
                        dec = -4.09933,
                        radius = 0.02)

In [9]:
# Output the table
gaia_table

solution_id,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_good_obs_ac,astrometric_n_bad_obs_al,astrometric_n_bad_obs_ac,astrometric_delta_q,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_primary_flag,astrometric_relegation_factor,astrometric_weight_al,astrometric_weight_ac,astrometric_priors_used,matched_observations,duplicated_source,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b,ecl_lon,ecl_lat,distance
str19,str19,str10,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,float64,bool,float64,float64,float64,int64,int64,bool,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,str13,float64,float64,float64,float64,float64
1635378410781933568,4365623394967062528,35099587,2015.0,254.28664594877367,0.7800591199324893,-4.099749987378444,0.7024372469056464,nan,nan,nan,nan,nan,nan,0.84535,nan,nan,nan,nan,nan,nan,nan,nan,nan,98,0,98,0,0,0,nan,1.924916386604309,92.79155731201172,False,7.47733736,0.2425749,nan,2,15,False,-7.87700462,-16.1296864,6.3257103,42.9588852,0.442392617,0.4329202,0.2614181,0.486563176,94,4185.569882100682,18.38994116052593,16.470383570620648,NOT_AVAILABLE,15.136960410817105,23.077171848464424,253.4558143667002,18.441611890550437,0.032923603904397254
1635378410781933568,4365623394967063808,687570591,2015.0,254.28780098630097,0.3397216821635168,-4.099498500282075,0.29689657486339727,nan,nan,nan,nan,nan,nan,0.86065,nan,nan,nan,nan,nan,nan,nan,nan,nan,88,0,87,0,1,0,nan,0.6602540016174316,16.458555221557617,False,2.73135948,1.64296556,nan,2,11,False,9.71254349,-10.5788441,22.3184643,-43.0938072,0.364626735,0.228746325,0.143637747,0.6820425,85,6263.45801028208,20.510147022314126,16.0327346381058,NOT_AVAILABLE,15.137853178927873,23.076325706082308,253.45699085708185,18.44199257620658,0.04899074978341256
1635378410781933568,4365623394967060608,880111999,2015.0,254.28611847638524,0.3561949355968105,-4.099302584519687,0.3601204958305138,nan,nan,nan,nan,nan,nan,0.68935,nan,nan,nan,nan,nan,nan,nan,nan,nan,79,0,77,0,2,0,nan,0.7493849992752075,25.84652328491211,False,3.05324078,1.44881618,nan,2,9,False,37.90309,32.5035744,32.8654747,41.38783,0.340221226,0.511677265,0.2953254,0.601127267,74,10414.377902601193,42.38322852209233,15.480686732076563,NOT_AVAILABLE,15.137071933590725,23.077854818529115,253.45520979204986,18.441996651890626,0.0527817395016619
1635378410781933568,4365623390673094528,328809522,2015.0,254.28630188415966,0.18787617975637025,-4.098723527834929,0.18040514452497075,nan,nan,nan,nan,nan,nan,0.659,nan,nan,nan,nan,nan,nan,nan,nan,nan,87,0,87,0,0,0,nan,0.3887620270252228,34.048095703125,False,1.916658,5.32621336,nan,2,11,False,29.7785,29.0432053,28.9352627,39.48811,0.420812249,0.5211736,0.332130939,0.5413987,83,35378.9074066953,39.64938494730562,14.152909022265671,NOT_AVAILABLE,15.13771157071593,23.07800482328448,253.45533206753862,18.44259273759549,0.055404535306989355
1635378410781933568,4365623394967139072,66578505,2015.0,254.28631458933617,5.190582688528432,-4.098362814717396,9.156621425650696,nan,nan,nan,nan,nan,nan,0.99775,nan,nan,nan,nan,nan,nan,nan,nan,nan,49,0,48,0,1,0,nan,0.7914198040962219,13.048690795898438,False,3.20657086,1.129101,nan,2,10,False,-12.55341,14.84307,7.092051,33.2416,0.6348926,0.508596659,0.406490982,0.3617657,49,

Even though the radius is specified in degrees, the resulting table lists it in arc minutes under the 'distance' column. To verify that the query above returns the correct observation, convert the distance values to degrees using the 'astropy.units' module and make sure they're less than the given radius.

In [10]:
(max(gaia_table['distance']) * u.arcmin).to(u.deg)

<Quantity 0.0199892 deg>

## Mast vs Observations

`Mast.mast_query` is similar to `Observations.query_criteria`. Both methods use keyword arguments to query MAST databases. All `astroquery.mast.Observations` class queries go through the CAOM  MAST database. Although users can similarly query CAOM with `Mast.mast_query`, there are some important differences and limitations between the two methods. The two examples below detail those differences by performing the same query using both methods. 

In [11]:
Observations.query_criteria(dataproduct_type=['image'],
                            proposal_pi='Osten*',
                            s_dec=[43.5, 45.5])

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str3,str7,str8,str3,str6,str7,str6,str18,str9,float64,float64,str5,str16,int64,float64,float64,float64,float64,float64,str6,float64,str4,str2,int64,str192,str1,str35,str6,bool,float64,str8,str9
science,HST,CALSTIS,STIS/CCD,HST,MIRVIS,Optical,EV-LAC,STAR;M V-IV;CORONA,o6au01msq,341.7067159883,44.33381826126,image,"Osten, Rachel A.",1,52172.687127777775,52172.68916674769,2.1,nan,nan,EV Lac,52537.68916657,8880,GO,--,POLYGON 341.7054021239983 44.333427657529214 341.70616680825816 44.3347273362981 341.7079843313322 44.33418022122444 341.70721964707235 44.332880542455555 341.7054021239983 44.333427657529214,--,mast:HST/product/o6au01msq_raw.fits,PUBLIC,False,nan,24476662,281780229
science,HST,CALSTIS,STIS/CCD,HST,MIRVIS,Optical,EV-LAC,STAR;M V-IV;CORONA,o6au01mtq,341.7067159813,44.33381825835,image,"Osten, Rachel A.",1,52172.69290408565,52172.692915659725,1.0,nan,nan,EV Lac,52537.69290498,8880,GO,--,POLYGON 341.7066757444726 44.33382271712645 341.70668339175285 44.33383571288162 341.70675621224444 44.3338137892895 341.70674856496413 44.33380079353434 341.7066757444726 44.33382271712645,--,mast:HST/product/o6au01mtq_raw.fits,PUBLIC,False,nan,24476663,281780226
science,HST,CALSTIS,STIS/CCD,HST,MIRVIS,Optical,EV-LAC,STAR;M V-IV;CORONA,o6au01muq,341.7067159741,44.33381825535,image,"Osten, Rachel A.",1,52172.694884108794,52172.69489568287,1.0,nan,nan,EV Lac,52537.69488411,8880,GO,--,POLYGON 341.7066757372726 44.333822714126455 341.70668338455283 44.333835709881626 341.7067562050444 44.333813786289504 341.7067485577641 44.33380079053434 341.7066757372726 44.333822714126455,--,mast:HST/product/o6au01muq_raw.fits,PUBLIC,False,nan,24476664,281780213


`Observations.query_criteria` is a simpler method for CAOM queries. It also allows wildcarding whereas `Mast.mast_query` requires exact string.

In [12]:
Mast.mast_query('Mast.Caom.Filtered',
                dataproduct_type = ['image'],
                proposal_pi = ['Osten, Rachel A.'],
                s_dec = [{'min':43.5, 'max':45.5}])

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str3,str7,str8,str3,str6,str7,str6,str18,str9,float64,float64,str5,str16,int64,float64,float64,float64,float64,float64,str6,float64,str4,str2,int64,str192,str1,str35,str6,bool,float64,str8,str9
science,HST,CALSTIS,STIS/CCD,HST,MIRVIS,Optical,EV-LAC,STAR;M V-IV;CORONA,o6au01muq,341.7067159741,44.33381825535,image,"Osten, Rachel A.",1,52172.694884108794,52172.69489568287,1.0,nan,nan,EV Lac,52537.69488411,8880,GO,--,POLYGON 341.7066757372726 44.333822714126455 341.70668338455283 44.333835709881626 341.7067562050444 44.333813786289504 341.7067485577641 44.33380079053434 341.7066757372726 44.333822714126455,--,mast:HST/product/o6au01muq_raw.fits,PUBLIC,False,nan,24476664,281780213
science,HST,CALSTIS,STIS/CCD,HST,MIRVIS,Optical,EV-LAC,STAR;M V-IV;CORONA,o6au01mtq,341.7067159813,44.33381825835,image,"Osten, Rachel A.",1,52172.69290408565,52172.692915659725,1.0,nan,nan,EV Lac,52537.69290498,8880,GO,--,POLYGON 341.7066757444726 44.33382271712645 341.70668339175285 44.33383571288162 341.70675621224444 44.3338137892895 341.70674856496413 44.33380079353434 341.7066757444726 44.33382271712645,--,mast:HST/product/o6au01mtq_raw.fits,PUBLIC,False,nan,24476663,281780226
science,HST,CALSTIS,STIS/CCD,HST,MIRVIS,Optical,EV-LAC,STAR;M V-IV;CORONA,o6au01msq,341.7067159883,44.33381826126,image,"Osten, Rachel A.",1,52172.687127777775,52172.68916674769,2.1,nan,nan,EV Lac,52537.68916657,8880,GO,--,POLYGON 341.7054021239983 44.333427657529214 341.70616680825816 44.3347273362981 341.7079843313322 44.33418022122444 341.70721964707235 44.332880542455555 341.7054021239983 44.333427657529214,--,mast:HST/product/o6au01msq_raw.fits,PUBLIC,False,nan,24476662,281780229


## Additional Resources

- [MAST Services](https://mast.stsci.edu/api/v0/_services.html) 
- [astroquery.mast Documentation](https://astroquery.readthedocs.io/en/latest/mast/mast_mastquery.html)

## Citations

If you use `astropy` or `astroquery` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`astroquery`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)



## About this Notebook

**Author(s):** Makayla St. Cyr <br>
**Keyword(s):** Tutorial, MAST, TESS, JWST, astroquery, cone <br>
**Last Updated:** Aug 2023 <br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 